<a href="https://colab.research.google.com/github/ItsShi/Mono-Depth/blob/main/SAM_ASF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# download

In [26]:
# !rm -rf /content/thesis
# !git clone https://github.com/ItsShi/thesis.git
!pip -q install timm tensorboardX==1.4 torchvision==0.12.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 1.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.11.0 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 1.11.0 which is incompatible.
torchtext 0.15.2 requires torch==2.0.1, but you have torch 1.11.0 which is incompatible.


In [6]:
# https://drive.google.com/file/d/1LrtAZ5NITeLYkS7Bsn-r7_OEzz8awizo/view?usp=sharing
import gdown

url = 'https://drive.google.com/uc?id=1LrtAZ5NITeLYkS7Bsn-r7_OEzz8awizo'
gdown.download(url,'weights_10.zip',quiet=True)
!unzip -q weights_10.zip

In [8]:
# https://drive.google.com/file/d/1zGXr3fTI-zdm_A9gViQrzzJ2qFvbhGN-/view?usp=sharing

url = 'https://drive.google.com/uc?id=1zGXr3fTI-zdm_A9gViQrzzJ2qFvbhGN-'
gdown.download(url,'weights_12.zip',quiet=True)
!unzip -q weights_12.zip

In [10]:
import gdown

perfect_gt_url = 'https://drive.google.com/uc?id=1GHX4ML7kUP4xFZSi56VtpZSxZdwHa2oO'
gdown.download(perfect_gt_url,'perfect_gt.zip',quiet=True)
!unzip -q perfect_gt.zip

import gdown

!mkdir videos
d3k1_url = 'https://drive.google.com/uc?id=17k1-CHptG_XTXVX8qp7yK_zadH54vVnY'
gdown.download(d3k1_url,'videos/d3k1_rgb.mp4',quiet=True)

!pip install ffmpeg

video_name = "d3k1"
!ffmpeg -i /content/videos/{video_name}_rgb.mp4 -filter:v "crop=1280:1024:0:0" /content/videos/{video_name}_crop_rgb.mp4
!ffmpeg -i /content/videos/{video_name}_crop_rgb.mp4 %6d.jpg
!mkdir -p /content/frames/{video_name}
!mv *.jpg /content/frames/{video_name}

import PIL.Image as pil
im = pil.open("/content/frames/{}/000001.jpg".format(video_name))
print(im.size)

  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6082 sha256=5b796eeb67defb1b0384c19befcf76ba076d3deefca3d3f511a2754eaee76a91
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --e

# test

In [31]:
# change image path in datasets/scared_dataset.py
def replace_text_in_file(search_path, replace_path, search_text, replace_text):
    with open(search_path, 'r') as file: content = file.read()
    for i in range(len(search_text)):
      modified_content = content.replace(search_text[i], replace_text[i])
      content = modified_content
    with open(replace_path, 'w') as file: file.write(modified_content)

# change splits/endovis/_files.txt
for opt in ["train", "val", "test"]:
    f = open("/content/thesis/splits/endovis/{}_files.txt".format(opt), "r+")
    # f = open("/content/{}_files_original.txt".format(opt), "r+")
    new_file = []
    for line in f:
      folder, frame = line.split()[0], line.split()[1]
      if int(frame)<20 and int(frame)>0:
        if folder == "dataset3/keyframe1" or folder == "dataset3/keyframe3" or folder == "dataset3/keyframe4":
          new_file.append(line)
    print(len(new_file))
    with open("{}_files.txt".format(opt), "w+") as f:
      for i in new_file:
        f.write(i)

    !cp -f /content/{opt}_files.txt /content/thesis/splits/endovis/{opt}_files.txt

16
16
2


In [33]:
!python thesis/train_end_to_end.py --no_cuda --scales 0 1 2 --data_path frames/d3k1 --log_dir sam  --batch_size 3 #--load_weights_folder womin/mdp/models/weights_19  --models_to_load "encoder" "depth" "pose_encoder" "pose"

[ 48  80 128] [0, 1, 2]
Training model named:
   mdp
Models and tensorboard events files are saved to:
   sam
Training is using:
   cpu
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:487: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Using split:
   endovis
There are 16 training items and 16 validation items

Training
/content/thesis/networks/depth_encoder.py:35: UserWarning: __floordiv__ is depr

# evaluate

In [ ]:
# _

In [44]:
!python thesis/evaluate_depth.py --eval_mono --data_path frames/d3k1 --eval_split endovis --load_weights_folder sam/mdp/models/weights_18

-> Loading weights from sam/mdp/models/weights_18
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:487: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
total samples:  2
batch size:  12
number of batch / epoch:  1
-> Computing predictions with size 320x256
/content/thesis/networks/depth_encoder.py:35: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch

In [37]:
!python thesis/evaluate_pose.py --data_path SCARED --eval_split endovis --load_weights_folder sam/mdp/models/weights_18

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:487: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
total samples:  833
batch size:  12
number of batch / epoch:  70
Traceback (most recent call last):
  File "/content/thesis/evaluate_pose.py", line 145, in <module>
    evaluate(options.parse())
  File "/content/thesis/evaluate_pose.py", line 97, in evaluate
    pose_encoder.cuda()
  File "/usr/local

In [35]:
!python thesis/test_simple.py --model_path sam/mdp/models/weights_18 --image_path SCARED --no_cuda --ext png

-> Loading model from  sam/mdp/models/weights_18
   Loading pretrained encoder
Traceback (most recent call last):
  File "/content/thesis/test_simple.py", line 138, in <module>
    test_simple(args)
  File "/content/thesis/test_simple.py", line 60, in test_simple
    encoder.load_state_dict(filtered_dict_enc)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1497, in load_state_dict
    raise RuntimeError('Error(s) in loading state_dict for {}:\n\t{}'.format(
RuntimeError: Error(s) in loading state_dict for ResnetEncoder:
	Missing key(s) in state_dict: "encoder.conv1.weight", "encoder.bn1.weight", "encoder.bn1.bias", "encoder.bn1.running_mean", "encoder.bn1.running_var", "encoder.layer1.0.conv1.weight", "encoder.layer1.0.bn1.weight", "encoder.layer1.0.bn1.bias", "encoder.layer1.0.bn1.running_mean", "encoder.layer1.0.bn1.running_var", "encoder.layer1.0.conv2.weight", "encoder.layer1.0.bn2.weight", "encoder.layer1.0.bn2.bias", "encoder.layer1.0.bn2.running